In [ ]:
import cv2
import torch
import os
import numpy as np
import pyttsx3
import random
from collections import defaultdict

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Load YOLOv5 model (small version for faster processing)
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Create a directory for saving frames if it doesn't exist
if not os.path.exists('results/frames'):
    os.makedirs('results/frames')

# Cooldown tracker to avoid repetitive commentary
cooldown_tracker = defaultdict(int)

# Function to detect entities (events) from the video
def detect_entities(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect objects in the current frame using YOLO
        results = yolo_model(frame)
        detections = results.xyxy[0].cpu().numpy()

        unique_labels = set()  # Track unique events for this frame
        commentary_pool = []  # Pool of commentary for this frame

        # Process each detected object
        for detection in detections:
            x1, y1, x2, y2, conf, cls = detection
            label = yolo_model.names[int(cls)]

            # Check if the label should be "hockey stick" based on heuristics
            if label == "baseball bat" or label == "sports ball":
                if is_hockey_stick(frame[int(y1):int(y2), int(x1):int(x2)]):
                    label = "hockey stick"

            team = classify_team(frame[int(y1):int(y2), int(x1):int(x2)])
            is_referee = classify_referee(frame[int(y1):int(y2), int(x1):int(x2)])

            # Aggregate unique events
            if label == "hockey stick" and "hockey stick" not in unique_labels:
                unique_labels.add("hockey stick")
                commentary_pool.append("A powerful shot with the hockey stick!")

            if team is not None and f"team_{team}" not in unique_labels:
                unique_labels.add(f"team_{team}")
                commentary_pool.append(f"A player from Team {team + 1} makes a move!")

            if is_referee and "referee" not in unique_labels:
                unique_labels.add("referee")
                commentary_pool.append("The referee is keeping a close eye on the game.")

        # Filter commentary to avoid consecutive repetitions
        filtered_commentary = []
        for comment in commentary_pool:
            if cooldown_tracker[comment] == 0:
                filtered_commentary.append(comment)
                cooldown_tracker[comment] = 5  # Cooldown period for this comment

        # Decrease cooldown for all phrases
        for key in list(cooldown_tracker.keys()):
            cooldown_tracker[key] = max(0, cooldown_tracker[key] - 1)

        # Generate audio commentary for the current frame
        if filtered_commentary:
            commentary_text = " ".join(filtered_commentary)
            print(f"Commentary: {commentary_text}")
            engine.say(commentary_text)
            engine.runAndWait()

        # Save the frame with bounding boxes
        save_frame(frame, frame_count)

        # Display the frame with bounding boxes
        cv2.imshow("Detected Entities", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

# Function to save the frame with bounding boxes
def save_frame(frame, frame_count):
    output_frame_path = f"results/frames/frame_{frame_count:04d}.jpg"
    cv2.imwrite(output_frame_path, frame)
    print(f"Saved frame {frame_count}.")

# Function to classify the team (based on color or other features)
def classify_team(player_image):
    hsv = cv2.cvtColor(player_image, cv2.COLOR_BGR2HSV)
    team_1_lower = np.array([100, 150, 50])
    team_1_upper = np.array([140, 255, 255])
    team_2_lower = np.array([0, 120, 70])
    team_2_upper = np.array([10, 255, 255])
    mask_team_1 = cv2.inRange(hsv, team_1_lower, team_1_upper)
    mask_team_2 = cv2.inRange(hsv, team_2_lower, team_2_upper)
    if cv2.countNonZero(mask_team_1) > cv2.countNonZero(mask_team_2):
        return 0
    elif cv2.countNonZero(mask_team_2) > cv2.countNonZero(mask_team_1):
        return 1
    else:
        return None

# Function to classify if the detected player is a referee
def classify_referee(player_image):
    gray = cv2.cvtColor(player_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, threshold1=100, threshold2=200)
    if np.count_nonzero(edges) > 500:
        return True
    return False

# Function to check if an object looks like a hockey stick
def is_hockey_stick(obj_image):
    height, width = obj_image.shape[:2]
    aspect_ratio = width / height
    if aspect_ratio > 4:
        return True
    return False

# Path to your video
video_path = 'clip.mp4'

print("Starting Entity Detection...")
detect_entities(video_path)
print("Entity Detection Completed!")


Using cache found in /home/prasanna-nage/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-27 Python-3.12.3 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/home/prasanna-nage/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Starting Entity Detection...
Commentary: The referee is keeping a close eye on the game. A player from Team 2 makes a move!


Saved frame 0.


/home/prasanna-nage/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/prasanna-nage/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved frame 1.
Saved frame 2.


...

Commentary: The referee is keeping a close eye on the game. A player from Team 2 makes a move!
Saved frame 40.
Saved frame 41.


/home/prasanna-nage/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/prasanna-nage/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
aplay: pcm_write:2146: write error: Interrupted system call
/home/prasanna-nage/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved frame 42.
Saved frame 43.
Saved frame 44.


/home/prasanna-nage/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/prasanna-nage/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Commentary: The referee is keeping a close eye on the game. A player from Team 2 makes a move!
